In [770]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stat
from sklearn import linear_model
from mpl_toolkits.mplot3d import Axes3D
import sklearn.model_selection 
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
import random
from sklearn.metrics import r2_score, mean_squared_error

In [771]:
import matplotlib.style as style
style.use('tableau-colorblind10')

In [772]:
if 'google.colab' in str(get_ipython()):
  from google_drive_downloader import GoogleDriveDownloader as gdd
  gdd.download_file_from_google_drive(file_id='1focpzp4j4-3Uierz8u2hK_nx1xG1qWvO',
  
  dest_path='./meatspec.csv')
else:
  print('You are not using Colab. Please define working_dir with the absolute path to the folder where you downloaded the data')

# Please modify working_dir only if you are using your Anaconda (and not Google Colab)
# You should write the absolute path of your working directory with the data
workingDir='./'

## 1) Preprocess the data

### 1) a) Set the radom seed to 0

In [773]:
np.random.seed(0)

### 1) b) Load the data. Print the mean, and standard deviation of every covariate. Is the data centered? Normalized? Standardized?

Answer 1) b) The data is not centered, since the means of covariates differ from 0. It's also not normalized since the max values of covariates is greater than 0, neither standardized since the standard deviation is different of 1.

In [774]:
path = './meatspec.csv'
meatspec = pd.read_csv(path)
meatspec.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V92,V93,V94,V95,V96,V97,V98,V99,V100,fat
0,2.61776,2.61814,2.61859,2.61912,2.61981,2.62071,2.62186,2.62334,2.62511,2.62722,...,2.98145,2.96072,2.94013,2.91978,2.89966,2.87964,2.85960,2.83940,2.81920,22.5
1,2.83454,2.83871,2.84283,2.84705,2.85138,2.85587,2.86060,2.86566,2.87093,2.87661,...,3.29186,3.27921,3.26655,3.25369,3.24045,3.22659,3.21181,3.19600,3.17942,40.1
2,2.58284,2.58458,2.58629,2.58808,2.58996,2.59192,2.59401,2.59627,2.59873,2.60131,...,2.68951,2.67009,2.65112,2.63262,2.61461,2.59718,2.58034,2.56404,2.54816,8.4
3,2.82286,2.82460,2.82630,2.82814,2.83001,2.83192,2.83392,2.83606,2.83842,2.84097,...,2.97367,2.94951,2.92576,2.90251,2.87988,2.85794,2.83672,2.81617,2.79622,5.9
4,2.78813,2.78989,2.79167,2.79350,2.79538,2.79746,2.79984,2.80254,2.80553,2.80890,...,3.30025,3.27907,3.25831,3.23784,3.21765,3.19766,3.17770,3.15770,3.13753,25.5


In [775]:
meatspec.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V92,V93,V94,V95,V96,V97,V98,V99,V100,fat
count,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,...,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000
mean,2.808561,2.811137,2.813727,2.816363,2.819098,2.821983,2.825064,2.828375,2.831943,2.835813,...,3.178262,3.158254,3.138534,3.119104,3.099971,3.081070,3.062290,3.043548,3.024895,18.142326
std,0.410793,0.413352,0.415906,0.418465,0.421040,0.423635,0.426245,0.428866,0.431510,0.434195,...,0.541957,0.541776,0.541519,0.541135,0.540563,0.539730,0.538586,0.537108,0.535354,12.740297
min,2.066420,2.065830,2.065180,2.064650,2.064170,2.063730,2.063400,2.063140,2.063010,2.063170,...,2.339720,2.320940,2.300430,2.280180,2.260580,2.241710,2.223520,2.206020,2.189130,0.900000
25%,2.512265,2.513260,2.514210,2.515330,2.516775,2.518240,2.518305,2.518605,2.519185,2.521480,...,2.781960,2.763715,2.741450,2.722130,2.702475,2.682635,2.664900,2.647370,2.628230,7.300000
50%,2.753600,2.755180,2.756680,2.758240,2.759860,2.761610,2.763550,2.765680,2.768660,2.770720,...,3.079400,3.058200,3.036290,3.014480,2.993020,2.971850,2.953740,2.935140,2.915640,14.000000
75%,3.006155,3.010470,3.014840,3.019260,3.025895,3.032780,3.039780,3.046930,3.054310,3.061875,...,3.493140,3.477830,3.462340,3.446320,3.429490,3.411360,3.393100,3.375965,3.358195,28.000000
max,4.237280,4.247210,4.257370,4.267730,4.278470,4.289680,4.301330,4.313310,4.325870,4.339270,...,5.128190,5.111870,5.095180,5.077600,5.058950,5.038260,5.015710,4.991070,4.965430,49.100000


### 1) c) Separate the data in train and test sets: save one fourth of the data as testing (you can use train_test_split from sklearn.model_selection) and standardize both the training and testing sets using the fit_transform and transform functions in sklearn.preprocessing.StandardScaler.

In [776]:
# Separationg the dataset into train and test sets
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(meatspec.loc[:, meatspec.columns != 'fat'], meatspec['fat'], test_size=0.25)

In [777]:
# Standardizing
from sklearn.preprocessing import StandardScaler
'''
DOCS OF sklearn.preprocessing.StandardScaler:

Centering and scaling happen independently on each feature by computing the relevant statistics on the samples in the training set. 
Mean and standard deviation are then stored to be used on later data using transform.
'''
scaler = StandardScaler()
covariates_train_standardized = scaler.fit_transform(X_train, y_train)
covariates_test_standardized = scaler.transform(X_test)
output_train_standardized = scaler.fit_transform(np.array(y_train).reshape(-1, 1))
output_test_standardized = scaler.transform(np.array(y_test).reshape(-1, 1))

print("Train covariates standardized: ")
pd.DataFrame(covariates_train_standardized).describe()

Train covariates standardized: 


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,...,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02,1.610000e+02
mean,-4.413309e-17,9.654113e-18,1.461909e-16,-4.978764e-16,-9.695488e-16,6.399298e-16,2.758318e-18,-2.978984e-16,-2.041155e-16,-4.992556e-16,...,2.872099e-16,-1.180905e-16,-2.723839e-17,3.616845e-16,3.137587e-16,6.237247e-16,-9.481718e-17,-3.203097e-16,1.565346e-16,-6.075196e-16
std,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,...,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00,1.003120e+00
min,-1.728379e+00,-1.725155e+00,-1.722144e+00,-1.718970e+00,-1.715868e+00,-1.712951e+00,-1.710172e+00,-1.707720e+00,-1.705466e+00,-1.703071e+00,...,-1.476519e+00,-1.474628e+00,-1.472811e+00,-1.474849e+00,-1.477329e+00,-1.479726e+00,-1.482024e+00,-1.484244e+00,-1.486276e+00,-1.488185e+00
25%,-6.951314e-01,-6.913551e-01,-6.874920e-01,-6.839169e-01,-6.826686e-01,-6.828912e-01,-6.831309e-01,-6.833389e-01,-6.834423e-01,-6.861470e-01,...,-7.668882e-01,-7.684970e-01,-7.701116e-01,-7.716562e-01,-7.730601e-01,-7.742044e-01,-7.751380e-01,-7.756372e-01,-7.757463e-01,-7.754645e-01
50%,-2.047058e-01,-2.064600e-01,-2.082710e-01,-2.100563e-01,-2.096814e-01,-2.092777e-01,-2.088319e-01,-2.082229e-01,-2.076415e-01,-2.068506e-01,...,-1.962268e-01,-1.971198e-01,-1.952131e-01,-1.898143e-01,-1.918213e-01,-1.942895e-01,-1.973396e-01,-2.009600e-01,-2.052609e-01,-2.102193e-01
75%,4.943333e-01,4.936166e-01,4.928318e-01,4.921155e-01,5.009920e-01,5.099344e-01,5.181839e-01,5.257000e-01,5.325583e-01,5.384968e-01,...,5.519996e-01,5.504754e-01,5.490854e-01,5.475408e-01,5.455876e-01,5.431792e-01,5.401710e-01,5.360391e-01,5.306937e-01,5.241516e-01
max,3.420805e+00,3.417746e+00,3.415238e+00,3.413037e+00,3.411344e+00,3.410190e+00,3.409433e+00,3.408775e+00,3.408630e+00,3.409288e+00,...,3.533737e+00,3.541486e+00,3.548788e+00,3.555476e+00,3.560998e+00,3.565404e+00,3.567641e+00,3.568526e+00,3.567899e+00,3.567193e+00


In [778]:
print("Test covariates standardized: ")
pd.DataFrame(covariates_test_standardized).describe()

Test covariates standardized: 


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,...,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,0.127194,0.127440,0.127686,0.127895,0.128073,0.128236,0.128404,0.128567,0.128736,0.128870,...,0.134913,0.134595,0.134303,0.134061,0.133880,0.133761,0.133731,0.133782,0.133892,0.134029
std,0.885122,0.885764,0.886349,0.886903,0.887404,0.887830,0.888174,0.888448,0.888656,0.888767,...,0.887192,0.886645,0.886144,0.885717,0.885410,0.885216,0.885254,0.885472,0.885853,0.886333
min,-1.430106,-1.427834,-1.425622,-1.423593,-1.421753,-1.419963,-1.419206,-1.419104,-1.418919,-1.418871,...,-1.232805,-1.230718,-1.228436,-1.225920,-1.223326,-1.220652,-1.218016,-1.215137,-1.212103,-1.208894
25%,-0.486316,-0.487349,-0.488307,-0.489389,-0.490569,-0.491719,-0.492804,-0.493889,-0.494980,-0.496143,...,-0.597700,-0.598400,-0.599236,-0.600101,-0.600973,-0.601890,-0.602750,-0.603643,-0.604515,-0.605451
50%,0.126253,0.122348,0.118658,0.115172,0.111965,0.109027,0.106407,0.104201,0.102444,0.101259,...,0.069284,0.067675,0.066003,0.064307,0.062651,0.061080,0.059545,0.058040,0.056553,0.059302
75%,0.500512,0.505092,0.509517,0.513791,0.517918,0.521810,0.525544,0.529207,0.532779,0.536195,...,0.694773,0.706775,0.719124,0.731236,0.739278,0.736948,0.734909,0.733074,0.731528,0.730169
max,2.538793,2.533538,2.528507,2.523692,2.518997,2.514403,2.509745,2.504793,2.499629,2.494405,...,2.686683,2.660846,2.635485,2.610375,2.586515,2.563434,2.542269,2.522663,2.504255,2.487269


In [779]:
print("Output train standardized: ")
pd.DataFrame(output_train_standardized).describe()

Output train standardized: 


,0
count,1.610000e+02
mean,2.206654e-17
std,1.003120e+00
min,-1.355039e+00
25%,-8.591285e-01
50%,-3.002452e-01
75%,7.860351e-01
max,2.391841e+00


In [780]:
print("Output test standardized: ")
pd.DataFrame(output_test_standardized).describe()

Output test standardized: 


,0
count,54.000000
mean,0.008788
std,1.011488
min,-1.268451
25%,-0.746958
50%,-0.343539
75%,0.697480
max,2.439070


## 1) d) Fit a regular OLS, do we need to fit the intercept?

As we can see, we don't need to fit the intercept, since both models predicted the variables with the same quadratic error. This is explainable since the data is centered the intercept is equal to 0. 

In [781]:
# With intercept
train = {'input': covariates_train_standardized, 'output': output_train_standardized}
OLS = LinearRegression(fit_intercept=True).fit(train['input'], train['output'])

# Testing 
pred_OLS = OLS.predict(covariates_test_standardized)

# Calculating the quadratic risk
r2_intercept = r2_score(output_test_standardized, pred_OLS)
r2_intercept

0.9600547777988199

In [782]:
# Without intercept
train = {'input': covariates_train_standardized, 'output': output_train_standardized}
OLS = LinearRegression(fit_intercept=False).fit(train['input'], train['output'])

# Testing 
pred = OLS.predict(covariates_test_standardized)

# Calculating the quadratic risk
r2_no_intercept = r2_score(output_test_standardized, pred)
r2_no_intercept

0.9600547777973001

## 1) e) Create a dataFrame df_coef and store the R2 coefficients of the estimated model. This data frame will be used along the TP to store and compare R2 coefficients of other variants of the OLS problem.

In [783]:
df_coef = pd.DataFrame([r2_no_intercept], columns=['R2'])
df_coef

,R2
0,0.960055


# 2) Program the method of the forward variable selection. You can use the test statistics of the test for nullity (as seen during the course). Do not define the stop criterion for the method, i.e. add a variables at each time until all the variables are used. Store the order of the variable selection and the associated p-value for each of them.

In [784]:
from scipy import stats
def forwardVarSelection(_x, _y):
    result = pd.DataFrame(columns=['covariates', 'p-score'])
    covariates = []
    p_values = []

    n = _x.shape[0] #number of samples 

    for outer_var in _x.columns:
        estimators = [] 
        T_values = []  
        for var in _x.columns:
            if var not in covariates:
                OLS = LinearRegression().fit(np.array(_x[var]).reshape(-1,1), _y)
                theta = np.array([OLS.intercept_, OLS.coef_[0]])
                estimators.append(theta)
    
                X_ = np.c_[np.ones(n), _x[var]]
          
                e = np.array([0,1])
                s_square = e.T @ np.linalg.inv( (X_.T @ X_)/n) @ e
                # In this last term we get the resiuals np.linalg.norm(_y - X_ @ theta)**2 / (n-2)
                T = np.sqrt(n)*np.linalg.norm(theta, ord=1) / (np.sqrt(s_square)*np.linalg.norm(_y - X_ @ theta)**2 / (n-2))
                T_values.append(float(T))
            else:
                estimators.append(np.array([0,0]))
                T_values.append(0)

        cov = _x.columns[T_values.index(max(T_values))]
        covariates.append(cov)

        p = 2*(1-stats.norm.cdf(max(T_values)))
        p_values.append(p)

        selected_X_ = np.c_[np.ones(n), _x[cov]]
        _y -=  selected_X_ @ np.array(estimators[cov])
        result = result.append({'covariates':'V' + str(int(cov) + 1), 'p-score': p}, ignore_index=True)
        result.sort_values(by='p-score', ascending=False)
    return result

In [785]:
FVS = forwardVarSelection(pd.DataFrame(covariates_train_standardized), pd.DataFrame(output_train_standardized))
FVS

,covariates,p-score
0,V41,0.000000
1,V8,0.017611
2,V40,0.017450
3,V7,0.016550
4,V42,0.017183
...,...,...
95,V75,0.928604
96,V77,0.945613
97,V76,0.971282
98,V50,0.977822


# 3) Run OLS on the variables with a p-value smaller than 0.05.

## (a) Apply the OLS of the sklearn library.

In [786]:
selected_variables = list(FVS[FVS['p-score'] < 0.05]['covariates'])
train_covariates = pd.DataFrame(covariates_train_standardized, columns = ['V' + str(elem) for elem in range(1, 101)])
train_covariates = train_covariates[selected_variables]
train_covariates

,V41,V8,V40,V7,V42,V9,V39,V6,V43,V10,...,V11,V98,V4,V44,V12,V36,V3,V97,V13,V99
0,2.142861,2.504793,2.149364,2.509745,2.144359,2.499629,2.161717,2.514403,2.154607,2.494405,...,2.489346,1.614927,2.523692,2.172471,2.483987,2.212972,2.528507,1.623595,2.478100,1.607894
1,-1.076830,-1.047651,-1.075191,-1.049498,-1.077394,-1.045949,-1.072719,-1.051324,-1.076723,-1.044446,...,-1.043088,-1.289928,-1.055025,-1.074853,-1.041928,-1.063234,-1.057061,-1.287228,-1.040905,-1.292497
2,-1.327322,-1.294725,-1.325165,-1.295192,-1.329661,-1.294324,-1.323209,-1.295956,-1.332081,-1.293943,...,-1.293608,-1.300876,-1.297732,-1.334384,-1.293543,-1.318014,-1.298628,-1.302829,-1.293679,-1.298921
3,-0.703804,-0.671902,-0.710795,-0.664212,-0.697781,-0.679216,-0.718063,-0.656124,-0.692922,-0.686147,...,-0.692655,-0.481069,-0.638679,-0.688702,-0.698767,-0.735810,-0.629373,-0.476371,-0.704416,-0.485731
4,-0.453418,-0.282486,-0.448510,-0.281648,-0.455541,-0.283533,-0.441581,-0.281040,-0.454604,-0.284701,...,-0.286038,-0.633257,-0.280430,-0.450909,-0.287727,-0.416298,-0.280240,-0.631515,-0.289811,-0.634540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,-0.534393,-0.490359,-0.532725,-0.492752,-0.534085,-0.488154,-0.529422,-0.495430,-0.531609,-0.486154,...,-0.484145,-0.690622,-0.501287,-0.527152,-0.482355,-0.515930,-0.504325,-0.688881,-0.480869,-0.691959
157,-0.737725,-0.652028,-0.732907,-0.652934,-0.741057,-0.651237,-0.727014,-0.653938,-0.742779,-0.650614,...,-0.650109,-1.051747,-0.656359,-0.743011,-0.649788,-0.708474,-0.657542,-1.049493,-0.649730,-1.053694
158,1.513459,1.396666,1.509751,1.396917,1.515313,1.396652,1.504838,1.397436,1.514720,1.396801,...,1.397130,1.561272,1.399467,1.511540,1.397759,1.488031,1.400537,1.564856,1.398717,1.557045
159,-1.511832,-1.505463,-1.515315,-1.506800,-1.507227,-1.504031,-1.517757,-1.508343,-1.501177,-1.502831,...,-1.501805,-1.300351,-1.512183,-1.493467,-1.501126,-1.519168,-1.514159,-1.301421,-1.500739,-1.298776


In [787]:
test_covariates = pd.DataFrame(covariates_test_standardized, columns = ['V' + str(elem) for elem in range(1, 101)])
test_covariates = test_covariates[selected_variables]

In [788]:
train = {'input': pd.DataFrame(train_covariates), 'output': pd.DataFrame(output_train_standardized)}
OLS = LinearRegression(fit_intercept=False).fit(train['input'], train['output'])

# Testing 
pred_FVS = OLS.predict(test_covariates)

## (b) Store the R2 coefficient in df_coef.

In [789]:
# Calculating the quadratic risk
r2 = r2_score(output_test_standardized, pred_FVS)
r2

0.9551797946016981

In [790]:
# Just to be sure df_coef is as it should be

# Adding R2 to df_coef
df_coef = df_coef.append(pd.DataFrame([r2], columns=['R2']), ignore_index=True)
df_coef

,R2
0,0.960055
1,0.955180


# 4) Using SequentialFeatureSelector on a linear regression estimator select (with forward selection), select the same number of variables as in the previous question.

## (a) Elaborate on why the 2 algorithms do not return the same variables and store the R2 onto the corresponding dataFrame.

As it's explained in the SequentialFeatureSelector docs, it uses cross-validaition methods to select the features, i.e. the method used to calculate the feature importance is different as so, the selected features are also different. The SequentialFeatureSelector uses by default a k-fold with k = 5 as cross-validation method without shuffle, so, if we change the value k we would also have a different feature selection:

In [791]:
from sklearn.feature_selection import SequentialFeatureSelector
selectedFeaturesOLS = SequentialFeatureSelector(LinearRegression(), n_features_to_select = len(selected_variables))
selectedFeaturesOLS.fit(covariates_train_standardized, output_train_standardized)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=23)

In [792]:
columns = selectedFeaturesOLS.get_support(indices=True) + 1
columns = ['V' + str(elem) for elem in list(columns)]
print('Selected covariates: ')
print(columns)
pd.DataFrame(selectedFeaturesOLS.transform(covariates_train_standardized), columns=columns)

Selected covariates: 
['V1', 'V2', 'V4', 'V8', 'V15', 'V19', 'V20', 'V22', 'V35', 'V40', 'V41', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V58', 'V75', 'V88', 'V91', 'V98', 'V100']


,V1,V2,V4,V8,V15,V19,V20,V22,V35,V40,...,V47,V48,V49,V50,V58,V75,V88,V91,V98,V100
0,2.538793,2.533538,2.523692,2.504793,2.464308,2.430295,2.419924,2.398447,2.230053,2.149364,...,2.241033,2.257382,2.267056,2.270310,2.169433,1.932944,1.737943,1.699024,1.614927,1.601872
1,-1.060982,-1.059052,-1.055025,-1.047651,-1.039633,-1.040526,-1.041226,-1.042500,-1.060251,-1.075191,...,-1.067218,-1.066165,-1.066595,-1.068603,-1.105113,-1.196138,-1.255938,-1.266923,-1.289928,-1.294878
2,-1.300573,-1.299531,-1.297732,-1.294725,-1.294345,-1.297608,-1.298699,-1.300848,-1.316598,-1.325165,...,-1.339710,-1.341072,-1.342720,-1.344953,-1.379374,-1.358263,-1.323744,-1.316112,-1.300876,-1.296990
3,-0.609860,-0.619777,-0.638679,-0.671902,-0.714071,-0.727493,-0.729678,-0.733519,-0.739507,-0.710795,...,-0.668040,-0.655861,-0.640443,-0.621400,-0.467655,-0.374494,-0.425644,-0.443951,-0.481069,-0.490111
4,-0.279992,-0.280183,-0.280430,-0.282486,-0.295186,-0.311146,-0.316127,-0.326455,-0.408205,-0.448510,...,-0.433658,-0.430391,-0.429867,-0.432137,-0.496426,-0.563418,-0.606897,-0.614754,-0.633257,-0.635455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,-0.510475,-0.507318,-0.501287,-0.490359,-0.479133,-0.480096,-0.481229,-0.483331,-0.511793,-0.532725,...,-0.510191,-0.506907,-0.505992,-0.507404,-0.543390,-0.613750,-0.663666,-0.672413,-0.690622,-0.693032
157,-0.660050,-0.658725,-0.656359,-0.652028,-0.650616,-0.656552,-0.658857,-0.663406,-0.703224,-0.732907,...,-0.743087,-0.746157,-0.751963,-0.760502,-0.860926,-0.970919,-1.024216,-1.033011,-1.051747,-1.055465
158,1.402745,1.401566,1.399467,1.396666,1.401520,1.411841,1.415634,1.424365,1.483533,1.509751,...,1.492629,1.488166,1.486449,1.487374,1.537744,1.593402,1.591347,1.583021,1.561272,1.552089
159,-1.518295,-1.516103,-1.512183,-1.505463,-1.500459,-1.502842,-1.503796,-1.505030,-1.518308,-1.515315,...,-1.462381,-1.450995,-1.439645,-1.428515,-1.368071,-1.314501,-1.305007,-1.303414,-1.300351,-1.296899


In [793]:
# The covariates that were found using both methods
intersection = []
for covariate in columns:
  if covariate in train_covariates:
    intersection.append(covariate)
intersection

['V4', 'V8', 'V40', 'V41', 'V98']

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
selectedFeaturesOLS = SequentialFeatureSelector(LinearRegression(), n_features_to_select = len(selected_variables), cv=2)
selectedFeaturesOLS.fit(covariates_train_standardized, output_train_standardized)

In [ ]:
columns = selectedFeaturesOLS.get_support(indices=True) + 1
columns = ['V' + str(elem) for elem in list(columns)]
print('Selected covariates: ')
print(columns)
pd.DataFrame(selectedFeaturesOLS.transform(covariates_train_standardized), columns=columns)

# 5) Code your own ridge estimator using expression derived in class. Test it for a penalty parameter α spaced evenly on a log scale 10e-9 to 10e2.

$ \theta^{rdg} = (X^TX + n \cdot \alpha \cdot I_p)^{-1}X^TY $

In [ ]:
# hat_est_ridge = (X^TX + n * alpha * I_p)^{-1} @ _x.T @ Y
def ridge_solver(_x, _y, _alpha):
  n = _x.shape[0]
  p = _x.shape[1]
  I_p = np.identity(p)
  return np.linalg.inv(_x.T @ _x + n * _alpha * I_p) @ _x.T @ _y

In [ ]:
alphas_ridge = np.logspace(-9, 2, 300)
for alpha in alphas_ridge:
  ridge_estimator = ridge_solver(covariates_train_standardized, output_train_standardized, alpha)
  pred = covariates_test_standardized @ ridge_estimator

## a) Plot how the values of the coefficients change with α

In [ ]:
# Ploting one estimator value

estimator_0 = []
fig, ax = plt.subplots()
for alpha in alphas_ridge:
  ridge_estimator = ridge_solver(covariates_train_standardized, output_train_standardized, alpha)
  estimator_0.append(ridge_estimator[0])
plt.plot(alphas_ridge, estimator_0)
plt.xscale('log')
plt.grid()
plt.title(r'Evolution of the value of one estimator within the evolution of $\alpha$')
plt.ylabel(r'$\theta_0$')
plt.ylabel(r'$\alpha$')
plt.show()

In [ ]:
# Ploting all estimator vector values

fig, ax = plt.subplots()
ridge_estimator = ridge_solver(covariates_train_standardized, output_train_standardized, 0)
for i in range(len(ridge_estimator)):
  estimator_0 = []
  for alpha in alphas_ridge:
    ridge_estimator = ridge_solver(covariates_train_standardized, output_train_standardized, alpha)
    estimator_0.append(ridge_estimator[i])
  plt.plot(alphas_ridge, estimator_0)
plt.xscale('log')
plt.title(r"Evolution of estimators' values within the evolution of $\alpha$")
plt.ylabel(r'$\theta_0$')
plt.ylabel(r'$\alpha$')
plt.grid()
plt.show()

## b) Plot how MSE of both the train and test sets change with α. Signal the minimum with a point.

In [ ]:
fig, ax = plt.subplots()
mse_train = []
mse_test = []
for alpha in alphas_ridge:
  ridge_estimator = ridge_solver(covariates_train_standardized, output_train_standardized, alpha)
  pred_test = covariates_test_standardized @ ridge_estimator
  pred_train = covariates_train_standardized @ ridge_estimator
  # MSE
  mse_train.append(mean_squared_error(output_train_standardized, pred_train))
  mse_test.append(mean_squared_error(output_test_standardized, pred_test))
plt.plot(alphas_ridge, mse_train)
plt.plot(alphas_ridge, mse_test)

plt.xscale('log')
plt.title(r'MSE over $\alpha$')
plt.ylabel(r'MSE')
plt.ylabel(r'$\alpha$')
plt.grid()
plt.legend(['Train', 'Test'])

## c) For the best performing value of α (the one with smallest training error) store the R2 results

In [ ]:
best_alpha_index = mse_train.index(min(mse_train))
best_alpha = alphas_ridge[best_alpha_index]
ridge_estimator = ridge_solver(covariates_train_standardized, output_train_standardized, best_alpha)
pred_train = covariates_train_standardized @ ridge_estimator
r2_best_alpha = r2_score(output_train_standardized, pred_train)

ridge_estimator = ridge_solver(covariates_test_standardized, output_test_standardized, best_alpha)
pred_test_ridge = covariates_test_standardized @ ridge_estimator

# 6) Use the sklearn version of the Lasso. Test it for a penalty parameter α spaced evenly on a log scale 10e-5 to 10e-2.

In [ ]:
estimators = []
mse_train = []
mse_test = []
alphas_lasso = np.logspace(-5, -2, 300)
for alpha in alphas_lasso:
    OLS = Lasso(alpha, tol=1e-1, max_iter=10e4).fit(covariates_train_standardized, output_train_standardized)
    pred_train = OLS.predict(covariates_train_standardized) 
    pred_test = OLS.predict(covariates_test_standardized)
    mse_train.append(mean_squared_error(output_train_standardized, pred_train))
    mse_test.append(mean_squared_error(output_test_standardized, pred_test))
    estimators.append(OLS.coef_)

## (a) To avoid having warnings and error you want to decrease the parameter tol or increase max_iter. Elaborate on why these warning arise and on the solution.

In the sklearn Lasso function we calculate the dual gap in order to find the optimal solution of the problem. As so, we stop searching whenever we reached the max_iter or the dual gap is smaller than tol. If we reach max_iter we receive this warning, and we don't receive the warning if we find a dual gap smaller than the tol before reaching max_iter. Therefore, if we increase the tol and/or increase the max_iter we will receive less oftenly warnings about the non-convergence

## b) Plot the number of coefficients that are different from 0 for each value of α

In [ ]:
#list of the number of coefficients that are different from 0
fig, ax = plt.subplots() 
coef_equal_0 = []
for estimator in estimators:
  coef_equal_0.append(len(estimator[estimator != 0]))
plt.plot(alphas_lasso, coef_equal_0)
plt.xscale('log')

## c) Plot how MSE of both the train and test sets change with α. Signal the minimum with a point.

In [ ]:
best_alpha_index_train_lasso = mse_train.index(min(mse_train))
best_alpha_index_test_lasso = mse_test.index(min(mse_test))
plt.plot(alphas_lasso, mse_train)
plt.plot(alphas_lasso, mse_test)
plt.plot(alphas_lasso[best_alpha_index_train_lasso], mse_train[best_alpha_index_train_lasso], 'o')
plt.plot(alphas_lasso[best_alpha_index_test_lasso], mse_test[best_alpha_index_test_lasso], 'o')
plt.legend(['Train', 'Test', 'Minimum MSE on train', 'Minimum MSE on test'])

In [ ]:
_alpha = alphas_lasso[best_alpha_index_train_lasso]
OLS = Lasso(_alpha, tol=1e-1, max_iter=10e4).fit(covariates_train_standardized, output_train_standardized)
pred_test_lasso = OLS.predict(covariates_test_standardized)

# 7) Code your own version of the crossvalidation. Preferable, in the same way as sklearn’s version, the length of every pair of folds should differ at most by one. Use the sklearn version of the Elastic net. Validate with a cross-validation that you implement. Test it for a penalty parameter α-ridge spaced evenly on a log scale 10e-10 to 10e3 and α-lasso in [0, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99].

# 8) For this question, we are going to use only variable 40 of the dataset original (non-centered) X. Plot the dataset and the regression line fitted with the whole sample. Generate 50 bootstrap samples, for each of the samples fit a regression model and plot the 50 estimated regression lines in the same plot (by setting alpha=.4 in the plotting function you can make the lines more transparent for the sake of readability of the plot). Finally, in the same plot, plot the prediction intervals (see exercise 12 in the lecture notes for the expression of the confidence intervals for the one dimensional case).

In [ ]:
from sklearn.utils import resample as bootstrap

In [ ]:
X_bootstrap = meatspec.iloc[:, 40].values.reshape(-1, 1)
Y_bootstrap = meatspec.iloc[:, -1].values.reshape(-1, 1)

In [ ]:
# OLS
OLS = LinearRegression(fit_intercept = True).fit(X_bootstrap,Y_bootstrap)
pred = OLS.predict(X_bootstrap)  
plt.scatter(X_bootstrap, Y_bootstrap)
plt.plot(X_bootstrap, pred, c='red')

In [ ]:
def bootstrap(_x, _y, b_samples):
    n = _x.shape[0]
    samples = []
    for i in range(0, b_samples):
        random_sample = np.random.choice(n, n, replace=True)
        samples.append([_x[random_sample], _y[random_sample]])
    return samples

In [ ]:
bootstrap_samples = bootstrap(X_bootstrap, Y_bootstrap, 50)
pd.DataFrame(bootstrap_samples).head()

In [ ]:
fig, ax = plt.subplots()
y_pred_values = []
mse_values = []
# CF = 1 - \alpha => alpha = 0.05
alpha = 0.05 

for each_sample in bootstrap_samples:   
    #bootstraping input and output
    X_sampled = each_sample[0]
    y_sampled = each_sample[1]
    
    OLS = LinearRegression(fit_intercept =True).fit(X_sampled,y_sampled)
    y_pred_bts = OLS.predict(X_sampled)  
    y_pred_values.append(y_pred_bts)

    plt.plot(X_sampled, y_pred_bts, alpha=.4)

n = X_bootstrap.shape[0]
gama = np.sqrt( (1/(n-2)) * np.sum(((Y_bootstrap-pred)**2)))

delta = np.mean(stats.t.ppf(1-alpha/2,n-2)*gama*np.sqrt(1+(1/n)))
upper_boundary = pred + delta
lower_boundary = pred - delta
y_pred_values = np.array(y_pred_values)

plt.plot(X_bootstrap, upper_boundary, color='C1')
plt.plot(X_bootstrap, lower_boundary, color='C1')

plt.scatter(X_bootstrap, Y_bootstrap)
plt.plot(X_bootstrap, pred, color='C2')

plt.grid()
plt.legend(['IC', 'Data', 'Model output'])


# 9) Compute the covariance matrix. Compute the singular value decomposition of the covariance matrix. For consistency in the notation use $ U, s, V = SVD(X^TX)$.

The covariance matrix of $X \in \mathbb{R}^{nxp}$ is given by $\Sigma = \frac{1}{n}X^TX$

In [ ]:
# The covariance matrix is a matrix giving the covariance between each pair of elements of a given random vector

X_PCA = meatspec.iloc[:, :-1]
Y_PCA = meatspec.iloc[:, -1]
scaler = StandardScaler()
X_PCA = scaler.fit_transform(X_PCA, Y_PCA)
Y_PCA = scaler.fit_transform(np.array(Y_PCA).reshape(-1, 1))
n = X_PCA.shape[0]
cov = X_PCA.T @ X_PCA / n
# SVD
U, s, V = np.linalg.svd(cov)
pd.DataFrame(cov)

## a) Plot a heatmap of the covariance matrix.

In [ ]:
import seaborn as sns
sns.heatmap(cov)

## (b) In PCA we transform the data to a new coordinate system such that the greatest variance by some scalar projection of the data lies on the first coordinate (called the first principal component, PC1), the second greatest variance in the second PC and so on. The PCs are computed given the above SVD, as XU. Instead of using the whole transformation, XU.

In [ ]:
PC = X_PCA @ U
pd.DataFrame(PC)

## c) Plot the amount of variance explained by the first k components for 
$k \in 2..p $

In [ ]:
# Computing the variance explained by each principal component
var_exp = (s ** 2) / sum(s**2)
plt.plot(np.arange(2, s.shape[0], 1), var_exp[2:s.shape[0]])

## d) We will use (as an approximation) the first 2 PCs. Plot the projected data using as color the value of y and interpret the plot.

In [ ]:
plt.scatter(X_PCA[:, :] @ PC[0, :], X_PCA[:, :] @ PC[1, :], c=Y_PCA)
plt.xlabel('Projection over PC1')
plt.ylabel('Projection over PC2')
plt.colorbar()

In [ ]:
plt.scatter(X_PCA[:, :] @ PC[0, :], np.zeros(len(X_PCA[:, :] @ PC[0, :])), c=Y_PCA)
plt.xlabel('Projection over PC1')
plt.colorbar()

In [ ]:
plt.scatter(X_PCA[:, :] @ PC[1, :], np.zeros(len(X_PCA[:, :] @ PC[1, :])), c=Y_PCA)
plt.xlabel('Projection over PC2')
plt.colorbar()

In [ ]:
plt.plot(X_PCA[:, :] @ PC[0, :])
plt.plot(X_PCA[:, :] @ PC[1, :])
plt.plot(Y_PCA)
plt.legend(['Projection with PC1', 'Projection with PC2', 'Y'])

In the graph 1, with the projections of the data over PC1 and PC2 we can see that mainly when the value of the projection over PC1 increases the points become also more yellow, i.e., y is also bigger, where we can see a clear relation between the labels and the points in the PC1. Also, for the PC2 we can see that it has a relation with PC1 and with y since as the projection over PC1 increases, the projection over PC2 decreases.

In [ ]:
k_index = np.arange(1, PC.shape[0], int(0.1*PC.shape[0]))
k_index

In [ ]:
X = pd.DataFrame()
for k in k_index:
  X = X.append(pd.DataFrame((X_PCA[:, :] @ PC[k, :]).reshape(-1, 1)).T)
X.T

In [ ]:
pd.DataFrame(Y_PCA.reshape(-1, 1))

In [ ]:
# Using PCS separately
r2_k_component = []
for k in k_index:
  #OLS = LinearRegression(fit_intercept=True).fit((X_PCA[:, :] @ PC[k, :]).reshape(-1, 1), Y_PCA.reshape(-1, 1))
  OLS = LinearRegression(fit_intercept=True).fit(X.T, Y_PCA.reshape(-1, 1))

  # Testing 
  #pred = OLS.predict((X_PCA[:, :] @ PC[k, :]).reshape(-1, 1))
  #r2_k_component.append(r2_score((X_PCA[:, :] @ PC[k, :]).reshape(-1, 1), pred))
  pred = OLS.predict(X.T)
  r2_k_component.append(r2_score(Y_PCA.reshape(-1, 1), pred))

# Adding R2 to df_coef
df_coef = df_coef.append(pd.DataFrame([max(r2_k_component)], columns=['R2']), ignore_index=True)
df_coef


In [ ]:
# name the rows
df_coef = df_coef.rename(index={0: 'Regular OLS', 1: 'OLS with FVS', 2: 'OLS with PCA'})
df_coef

## 10) Summarize the results of the models and elaborate in their main characteristics.

OLS assumes linearity and normally distributed variables, Ridge and Lasso help to handle multicollinearity and prevent overfitting by adding penalty terms to the sum of squared residuals. Although Ridge converged to a great regression (evaluated by its r2_score), a bad choice in alpha would lead in bad fitting, so we iterated through a range of alphas and picked the best. In this dataset, smaller alphas lead to better fitting models, which indicates small collinearity between covariates.

We also applied a forward variable selection before using OLS, which lead to a significant reduction on the amount of used variables and a model with great $R²$ score.

We can see that in our Lasso model as the regularization term increased, more estimators were set to 0. In this sense, we could use Lasso for feature selection.

The bootstrap resampling process allowed some variation inside a confidence interval in our regression models. This could be used to analyze the distribution of our coefficients.

OLS with PCA reduces the number of features in the data using PCA, which can help to prevent overfitting and reduce computational complexity. It caused a significant reduction on the $R²$ and we also arrived at a good $R²$ score.

In [ ]:
plt.figure(figsize=(10,5))

# OLS
plt.plot(pred_OLS)

# Forward variable selection and OLS
plt.plot(pred_FVS)

# Ridge 
plt.plot(pred_test_ridge)

# Lasso
plt.plot(pred_test_lasso)

# real data
plt.plot(output_test_standardized)


plt.title('Predictions using different regression models')
plt.grid()
plt.legend(['Regular OLS', 'OLS with FVS', 'Ridge', 'Lasso', 'Real output'])

In [ ]:
df_coef